# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
from pathlib import Path
import hvplot.pandas
# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")
import geoviews as gv
import pyproj
import cartopy.crs as ccrs

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,glace bay,46.1969,-59.9570,271.51,66,75,11.83,CA,1704820665
1,1,xifeng,42.7372,124.7222,255.46,98,15,1.76,CN,1704820667
2,2,bomdila,27.2500,92.4000,277.88,83,98,0.55,IN,1704820668
3,3,longyearbyen,78.2186,15.6401,259.06,72,95,9.26,SJ,1704820669
4,4,taoudenni,22.6783,-3.9836,300.19,15,98,2.64,ML,1704820671


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
# YOUR CODE HERE
map_plot_1 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (
    (city_data_df['Max Temp'] > 294.15) & 
    (city_data_df['Max Temp'] < 300.15) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
)

filtered_cities = city_data_df[ideal_conditions]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
211,211,isangel,-19.5500,169.2667,299.11,84,0,3.73,VU,1704821001
231,231,tamanrasset,22.7850,5.5228,295.10,13,0,2.06,DZ,1704821034
409,409,pisco,-13.7000,-76.2167,297.60,78,0,3.60,PE,1704821323
550,550,sur,22.5667,59.5289,295.03,43,0,3.58,OM,1704822749


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''
# Display sample data
# YOUR CODE HERE
print(hotel_df.head())


            City Country      Lat       Lng  Humidity Hotel Name
211      isangel      VU -19.5500  169.2667        84           
231  tamanrasset      DZ  22.7850    5.5228        13           
409        pisco      PE -13.7000  -76.2167        78           
550          sur      OM  22.5667   59.5289        43           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation",
    "apiKey": geoapify_key,
    "limit":5
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print (hotel_df)

Starting hotel search
isangel - nearest hotel: Tanna Lodge
tamanrasset - nearest hotel: فندق الأمان
pisco - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
            City Country      Lat       Lng  Humidity      Hotel Name
211      isangel      VU -19.5500  169.2667        84     Tanna Lodge
231  tamanrasset      DZ  22.7850    5.5228        13     فندق الأمان
409        pisco      PE -13.7000  -76.2167        78  No hotel found
550          sur      OM  22.5667   59.5289        43       Sur Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=['Country','Hotel Name']
)
# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)